In [5]:
import pandas as pd
import clickhouse_connect
from datetime import datetime



# Import relevant functionality

from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from pandas.core.interchange.dataframe_protocol import DataFrame

load_dotenv()
# Создаём память
memory = MemorySaver()
from langchain_core.tools import tool

# Инициализируем OpenAI-модель
# llm = init_chat_model("gpt-4.1-mini", model_provider="openai")
llm = init_chat_model("gpt-5", model_provider="openai")

# llm = init_chat_model("gpt-4.1", model_provider="openai")
# llm = init_chat_model("o3-mini", model_provider="openai")  # можно и так

# Подключаем инструмент поиска
search = TavilySearch(max_results=2)


In [2]:
CH_HOST = '84.201.160.255'   # если подключаешься через SSH-туннель (рекомендую)
CH_PORT = 8123         # локальный порт туннеля для HTTP (8123 на сервере → 18123 локально)
CH_USER = 'peter'   # пользователь ClickHouse
CH_PASS = '1234'      # пароль пользователя (если пустой — оставь '')

In [3]:


# создаём клиент
client = clickhouse_connect.get_client(
    host="84.201.160.255",   # замени на свой хост/адрес
    port=8123,
    username="peter",
    password="1234"
)

# # готовим запись
# data = {
#     "id": ["1"],
#     "message_id": ["msg-001"],
#     "subject": ["Test subject"],
#     "from_addr": [["sender@example.com"]],   # массив строк -> внутри список
#     "to_addr": [["receiver2@example.com"]],
#     "cc_addr": [[]],                         # пустой массив
#     "bcc_addr": [[]],
#     "sent_at_utc": [datetime.utcnow()],
#     "sent_at_raw": ["Mon, 26 Aug 2025 14:00:00 +0000"],
#     "folder": ["Inbox"],
#     "body_text": ["Hello, this is message for test."],
#     "body_html": ["<p>Hello, this is <b>HTML</b>.</p>"]
# }
# 
# df = pd.DataFrame(data)
# 
# # пушим в ClickHouse
# client.insert_df(
# 
#     "mailkb.emails",
#     df
# )


In [4]:
# SQL-запрос
query = "SELECT * FROM mailkb.emails LIMIT 100"

# вернуть сразу DataFrame
df = client.query_df(query)


In [5]:
df

,id,message_id,subject,from_addr,to_addr,cc_addr,bcc_addr,sent_at_utc,sent_at_raw,folder,body_text,body_html
0,archive.pst::2099268,,,[],[],[],[],1970-01-01,,unknown,Ostrik Petr 11:17: \r\n\r\nздравствуйте\r\n\r\...,"<html xmlns=""http://schemas.microsoft.com/2008..."
1,archive.pst::2105796,,,[],[],[],[],1970-01-01,,unknown,,
2,archive.pst::2106116,,,[],[],[],[],1970-01-01,,unknown,Cherkesova Olesya 21:50: \r\n\r\nПетя\r\n\r\nт...,"<html xmlns=""http://schemas.microsoft.com/2008..."
3,archive.pst::2115844,,,[],[],[],[],1970-01-01,,unknown,Пропущенный звонок от пользователя Katushenko ...,"<html xmlns=""http://schemas.microsoft.com/2008..."
4,archive.pst::2118532,,,[],[],[],[],1970-01-01,,unknown,,
...,...,...,...,...,...,...,...,...,...,...,...,...
95,archive.pst::2233220,,,[],[],[],[],1970-01-01,,unknown,"Алина, добрый день, совсем чуть-чуть дополнил ...",
96,archive.pst::2233252,,,[],[],[],[],1970-01-01,,unknown,"Добрый день, нет.\r\n\r\nА от кого она должна ...","<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."
97,archive.pst::2233284,,,[],[],[],[],1970-01-01,,unknown,Добрый день!\r\n\r\n \r\n\r\nПрошу перенести з...,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."
98,archive.pst::2233316,,,[],[],[],[],1970-01-01,,unknown,\r\n\r\n \r\n\r\nFrom: Fedorov Aleksey [mailt...,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ..."


In [10]:
df.columns

Index(['id', 'message_id', 'subject', 'from_addr', 'to_addr', 'cc_addr',
       'bcc_addr', 'sent_at_utc', 'sent_at_raw', 'folder', 'body_text',
       'body_html'],
      dtype='object')

In [9]:
@tool
def deep_analysis(query: str) -> str:
    """
    write sql query to get information from table mailkb.emails in clickhouse.
    The table has the floowing stucture:
    -- mailkb.emails определение

    (
    
        `id` String,
    
        `message_id` String, 
    
        `subject` String,
    
        `from_addr` Array(String),
    
        `to_addr` Array(String),
    
        `cc_addr` Array(String),
    
        `bcc_addr` Array(String),
    
        `sent_at_utc` DateTime64(3,
     'UTC'),
    
        `sent_at_raw` String,
    
        `folder` String,
    
        `body_text` String,
    
        `body_html` String
    )

     Take first 100 mails. You need to analize body_text and try to categorize mails basing on topics disscussed
    """
    print(query)

    df = client.query_df(query) 
    return df.to_json(orient="records")

In [10]:
tools = [deep_analysis,search]

# Создаём агента
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [8]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "what themes are in mails?",
}
for step in agent_executor.stream(
        {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what themes are in mails?
================================== Ai Message ==================================

The emails mainly discuss professional topics including:

1. SAP ERP and Transportation Management System (TMS) integration issues and troubleshooting.
2. Incident tracking and resolution related to SAP system errors and support cases.
3. Technical support communications regarding configuration, debugging, and error analysis in enterprise systems.
4. Sharing of project documents, screenshots, and technical files.
5. Internal collaboration between consultants, analysts, and project managers about tasks and project progress.
6. Notifications from SAP support and system updates.
7. Queries and solutions about transportation orders, invoice processing, and master data synchronization.
8. Coordination of work schedules and team communications.

Overall, the themes revolve around enterprise software suppo

### улучшение промта. вариант 1

In [11]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "what themes are in mails?",
}

input_message_system = {
    "role": "system",
    "content": """ try to define to which project mails are belong to. If you have doubts,  choose project named "other".
    After try to find main problems/subjects in every project and people which are involved. After, for every problem/subject find conclusion and final status of the topic.""",
}

for step in agent_executor.stream(
        {"messages": [input_message_system, input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what themes are in mails?
================================== Ai Message ==================================
Tool Calls:
  deep_analysis (call_ydpqvWD2gIEgkMsWpi8O4Acz)
 Call ID: call_ydpqvWD2gIEgkMsWpi8O4Acz
  Args:
    query: SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr, sent_at_utc, folder, body_text FROM mailkb.emails ORDER BY sent_at_utc DESC LIMIT 100;
SELECT id, message_id, subject, from_addr, to_addr, cc_addr, bcc_addr, sent_at_utc, folder, body_text FROM mailkb.emails ORDER BY sent_at_utc DESC LIMIT 100;
================================= Tool Message =================================
Name: deep_analysis

[{"id":"1","message_id":"msg-001","subject":"Test subject","from_addr":["sender@example.com"],"to_addr":["receiver2@example.com"],"cc_addr":[],"bcc_addr":[],"sent_at_utc":1756398,"folder":"Inbox","body_text":"Hello, this is message for test."},{"id":"1","message_id":"msg-00

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-5 in organization org-bGgejPh56Ldic32xb1X7KBBY on tokens per min (TPM): Limit 500000, Requested 670496. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

### QUADRANT

In [4]:
load_dotenv()

NameError: name 'load_dotenv' is not defined

In [3]:
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
client_qd = QdrantClient(url=QDRANT_URL)

NameError: name 'QDRANT_URL' is not defined

In [12]:
qdrant = QdrantVectorStore(
            client=client_qd,
            collection_name=folder,
            embedding=embeddings,
            # vector_name="vector",  # uncomment if your collection uses a named vector
        )

NameError: name 'QdrantVectorStore' is not defined

In [18]:
@tool
def deep_analysis(query: str) -> str:
    """
    Write a single ClickHouse SQL query over table mailkb.emails.
    Return only recent, non-empty bodies and a normalized subject.

    Required columns to select:
      id, message_id,
      replaceRegexpAll(lower(subject),'^(re:|fw:|fwd:)\\s*','') AS subject_norm,
      from_addr, to_addr, cc_addr, bcc_addr,
      sent_at_utc, folder, body_text

    Filters:
      body_text != '' AND lengthUTF8(body_text) >= 50

    Ordering/limit:
      ORDER BY sent_at_utc DESC LIMIT 300

    The tool will execute the SQL and return JSON records.
    """
    print(query)
    df = client.query_df(query)
    return df.to_json(orient="records")


In [19]:
tools = [deep_analysis,search]

# Создаём агента
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [19]:
import json, time, math, random
from collections import defaultdict

BATCH_SIZE = 50
MAX_EMAILS = 600  # общая выборка на прогон
OFFSETS = list(range(0, MAX_EMAILS, BATCH_SIZE))

def mk_sql(limit, offset):
    return f"""
SELECT
  id,
  message_id,
  replaceRegexpAll(lower(subject), '^(re:|fw:|fwd:)\\s*', '') AS subject_norm,
  from_addr, to_addr, cc_addr, bcc_addr,
  sent_at_utc,
  folder,
  left(body_text, 800) AS body_text_snip
FROM mailkb.emails
WHERE body_text != '' AND lengthUTF8(body_text) >= 50
ORDER BY sent_at_utc DESC
LIMIT {limit} OFFSET {offset};
""".strip()

# Промпт для батча (минималистичный)
def batch_prompt(sql, batch_size):
    return {
        "role":"user",
        "content": f"""Task: discover themes in this BATCH of Outlook emails (<= {batch_size}).

Step 1 — fetch data:
Call tool deep_analysis with EXACTLY this SQL:
{sql}

Step 2 — analyze only the returned JSON. 
Make 5–8 themes max. Respond ONLY JSON array:
[{{"theme": str, "keywords":[str,...], "count": int, "example_subjects":[str,...]}}]

Rules:
- Keep it under ~600 tokens.
- No prose outside JSON.
- If uncertain, fewer themes with broader keywords."""
    }

def retry_backoff(call, *, retries=5, base=1.5, jitter=0.25):
    for i in range(retries):
        try:
            return call()
        except Exception as e:
            msg = str(e).lower()
            if "rate_limit" in msg or "429" in msg or "tpm" in msg:
                sleep = (base ** i) + random.uniform(0, jitter)
                time.sleep(sleep)
                continue
            raise
    raise RuntimeError("Exceeded retries due to rate limits")

def run_batch(agent_executor, config, sql):
    im = batch_prompt(sql, BATCH_SIZE)
    def _call():
        out = agent_executor.invoke({"messages":[im]}, config)
        return out["messages"][-1].content
    return retry_backoff(_call)

def merge_theme_summaries(partials):
    # Простое объединение: одинаковое имя темы → суммируем счётчики
    # (на следующих итерациях можно добавить фуззи-слияние по ключевым словам)
    merged = {}
    for part in partials:
        try:
            arr = json.loads(part)
        except:
            continue
        for t in arr:
            name = (t.get("theme") or "").strip().lower()
            if not name:
                continue
            if name not in merged:
                merged[name] = {
                    "theme": t.get("theme"),
                    "keywords": set(t.get("keywords") or []),
                    "count": int(t.get("count") or 0),
                    "example_subjects": list(t.get("example_subjects") or [])[:3]
                }
            else:
                merged[name]["count"] += int(t.get("count") or 0)
                merged[name]["keywords"].update(t.get("keywords") or [])
                ex = merged[name]["example_subjects"] + (t.get("example_subjects") or [])
                merged[name]["example_subjects"] = ex[:3]
    # to list + сортировка
    res = []
    total = sum(v["count"] for v in merged.values()) or 1
    for k,v in merged.items():
        v["keywords"] = sorted(list(v["keywords"]))[:10]
        v["share"] = round(v["count"] / total, 2)
        res.append(v)
    res.sort(key=lambda x: x["count"], reverse=True)
    return res, total


In [20]:
input_message = {
    "role": "user",
    "content": """You are doing MAP-REDUCE theming.

Constraints:
- Work in batches of 50 emails (LIMIT/OFFSET).
- Each batch reply is JSON array only, <= 600 tokens, 5–8 themes.
- Use body_text_snip only (<=800 chars), ignore HTML.
- No prose outside JSON.

Now process the first batch."""
}
